In [1]:
import json
from pathlib import Path

schema_path = Path("business_plan_schema.json")
with schema_path.open(encoding="utf-8") as f:
    schema = json.load(f)

In [2]:
schema_keys = list(schema.keys())

general_information = schema_keys[:4]
general_information_team_composition= schema_keys[5:6]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 전문 사업 컨설턴트이자 사업계획서 작성 전문가이다. 스타트업, 중소기업, 투자 유치용 사업계획서 작성에 특화되어 있으며, 사용자가 제공하는 정보와 요구사항을 바탕으로 체계적이고 설득력 있는 사업계획서를 작성하는 역할을 수행한다.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "user",
            '''스키마에서 이번에 채워야 할 필드는 **{general_information}**이다.  
관련해서 회사DB와 공고문DB에서 검색된 자료는 다음과 같다:  

- 회사DB: 사업명:제로제로, 대표명:장정호
- 공고문DB: None


위 정보를 참고하여 {request}'''
        ), 
    ]
)

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    # model="gpt-5-nano"
    model="gpt-5"
)

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

In [5]:
from langchain_core.output_parsers import StrOutputParser

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory

# 세션 기록을 저장할 딕셔너리
store = {}

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="request",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [8]:
import uuid

session_id = uuid.uuid4()

chain_with_history.invoke(
    # 질문 입력
    {"request": f"**{general_information}** 항목을 작성하라.", "general_information": general_information},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": session_id}},
)

[대화 세션ID]: 8668fc18-70a3-4cf3-a10f-f3ff9450f547


'다음과 같이 채워드립니다.\n\n- item_name: 제로제로\n- deliverables:\n  - 사업개요 및 목표 정의\n  - 시장 및 경쟁 분석\n  - 비즈니스 모델 및 수익화 전략\n  - 마케팅/영업 전략 및 초기 고객 확보 계획\n  - 운영계획 및 조직구조\n  - 재무예측(손익계산서/현금흐름/자본요구 분석)\n  - 위험분석 및 대응전략\n  - 실행 로드맵 및 KPI\n  - 투자제안서 요약 및 피치덱 초안\n- job: 대표이사(CEO)\n- company_name: 제로제로'

In [19]:
chain_with_history.invoke(
    # 질문 입력
    {"test": "","general_information": ["name", "category"]},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": session_id}},
)

[대화 세션ID]: 2b400381-0c9c-4c41-bbbd-aca830335163


'- name: 제로제로\n- category: 투자유치용 사업계획서 대상 기업(스타트업)\n\n비고: 공고문DB가 None이므로 카테고리 정보를 추정적으로 기입했습니다. 산업군 등 더 구체적인 카테고리가 필요하면 알려주시면 업데이트하겠습니다.'

In [21]:
list_a = [1,2,3]
str(list_a)

'[1, 2, 3]'

In [13]:
# from langchain_community.llms import VLLMOpenAI

# llm = VLLMOpenAI(
#     openai_api_key="EMPTY",
#     openai_api_base="http://localhost:8000/v1",
#     model_name="K-intelligence/Midm-2.0-Base-Instruct",
#     model_kwargs={"stop": ["."]},
# )

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v1",
    model="K-intelligence/Midm-2.0-Base-Instruct",
)

In [10]:
from typing import List
from pydantic import BaseModel, Field

class GeneralMember(BaseModel):
    """팀 구성 현황(대표자 본인 제외)의 한 행"""
    position: str = Field(default="", description="직위 (예: 공동대표, 대리 등)")
    task: str = Field(default="", description="담당 업무 (예: S/W 개발 총괄, 홍보 및 마케팅 등)")
    capability: str = Field(default="", description="보유 역량 (예: OO학 학사, OO 관련 경력(00년 이상))")
    status: str = Field(default="", description="구성 상태 (예: 완료, 예정 등)")

class GeneralInfo(BaseModel):
    """사업계획서 - 일반현황 섹션"""
    item_name: str = Field(default="", description="사업아이템명 (예: OO기술이 적용된 OO기능의(혜택을 제공하는) OO제품·서비스 등)")
    deliverables: str = Field(default="", description="산출물(협약기간 내 목표) (예: 모바일 어플리케이션(0개), 웹사이트(0개))")
    job: str = Field(default="", description="직업(직장명 기재 불가) (예: 교수 / 연구원 / 사무직 / 일반인 / 대학생 등)")
    company_name: str = Field(default="", description="기업(예정)명 기업명이 없다면 '한국IT'라고 작성")
    general_members: List[GeneralMember] = Field(
        default_factory=list,
        description="팀 구성 현황 리스트(대표자 본인 제외)"
    )

In [14]:
from typing import List
from pydantic import BaseModel, Field

class GeneralMember(BaseModel):
    """팀 구성 현황(대표자 본인 제외)의 한 행"""
    position: str = Field(default="", description="직위")
    task: str = Field(default="", description="담당 업무")
    capability: str = Field(default="", description="보유 역량")
    status: str = Field(default="", description="구성 상태")

class GeneralInfo(BaseModel):
    """사업계획서 - 일반현황 섹션"""
    item_name: str = Field(default="", description="사업아이템명")
    deliverables: str = Field(default="", description="산출물(협약기간 내 목표)")
    job: str = Field(default="", description="직업(직장명 기재 불가)")
    company_name: str = Field(default="", description="기업(예정)명")
    general_members: List[GeneralMember] = Field(
        default_factory=list,
        description="팀 구성 현황 리스트(대표자 본인 제외)"
    )

In [15]:
structured_llm = llm.with_structured_output(GeneralInfo)

In [16]:
structured_llm.invoke(
'''
RAG이용 사업 계획서의 초안을 만들어라

사업아이템명 (예: OO기술이 적용된 OO기능의(혜택을 제공하는) OO제품·서비스 등)
산출물(협약기간 내 목표) (예: 모바일 어플리케이션(0개), 웹사이트(0개))
직업(직장명 기재 불가) (예: 교수 / 연구원 / 사무직 / 일반인 / 대학생 등)
기업(예정)명 기업명이 없다면 '한국IT'라고 작성

팀 구성 현황 리스트(대표자 본인 제외)
직위 (예: 공동대표, 대리 등)
담당 업무 (예: S/W 개발 총괄, 홍보 및 마케팅 등)
보유 역량 (예: OO학 학사, OO 관련 경력(00년 이상))
구성 상태 (예: 완료, 예정 등)
'''
)

GeneralInfo(item_name='AI 기반 맞춤형 건강 관리 플랫폼', deliverables='모바일 어플리케이션 1개, 웹 대시보드 1개', job='연구원', company_name='한국IT', general_members=[GeneralMember(position='공동대표', task='비즈니스 전략 수립 및 운영 관리', capability='경영학 석사, 스타트업 경영 5년 이상', status='완료'), GeneralMember(position='소프트웨어 개발자', task='플랫폼 개발 및 기술 지원', capability='컴퓨터공학 학사, 백엔드 개발 3년 이상', status='예정'), GeneralMember(position='데이터 분석가', task='데이터 분석 및 인사이트 제공', capability='통계학 석사, 데이터 분석 2년 이상', status='예정'), GeneralMember(position='마케팅 전문가', task='홍보 및 시장 진입 전략', capability='광고학 학사, 디지털 마케팅 4년 이상', status='예정')])

test_llm = 

In [18]:
from langchain_community.llms import VLLMOpenAI

llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v1",
    model_name="K-intelligence/Midm-2.0-Base-Instruct",
    model_kwargs={"stop": ["."]},
)
print(llm.invoke("Rome is"))

 known as the Eternal City and for good reason
